In [86]:
!pip install elasticsearch[async]==7.17.0 -q
!pip install elasticsearch-dsl==7.4.0 -q

In [25]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd
import ast
import json

In [26]:
class ESTag:
    INLINE_CHOICE = "INLINE_CHOICE"
    INLINE_ANSWER = "INLINE_ANSWER"

In [27]:
client = Elasticsearch("http://35.224.116.253:7032")
index = "fluentd-*"

In [28]:
# client.delete_by_query(index=[index], body={"query": {"match_all": {}}}, size=1000000)

In [29]:
client.info()

{'name': 'elastic',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'QeWlxU4zT4STOl7p7e6P9g',
 'version': {'number': '7.0.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'e4efcb5',
  'build_date': '2019-04-29T12:56:03.145736Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [30]:
answ_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_ANSWER, fields=['log'])
choice_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_CHOICE, fields=['log'])

In [31]:
#cand_id - is guid. Example: 3940172d-fda5-4e02-b284-6f5e2074454c 
def find_choice_for_answer(cand_id, choices):
    for i in choices:
        choice_id = i['update']['result_id'].split('__')[1]
        if choice_id == cand_id:
            return i

In [32]:
choice_logs = []
rows = []

for hit in choice_search.scan():
    _hit = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit['update'] = json.loads(_hit['update'])
    _hit["timestamp"] = hit["@timestamp"]
    choice_logs.append(_hit)

for hit in answ_search.scan():
    result_row = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit["answer_timestamp"] = hit["@timestamp"]
    _hit_template_uniq_id = _hit['template_options'][0].split('__')[1]
    possible_choice = find_choice_for_answer(_hit_template_uniq_id, choice_logs)
    update = json.loads(_hit['update'])
    
    if possible_choice is not None:
        choice_update = possible_choice['update']
        result_row["template_choice"] = choice_update["result_id"]
        result_row["choice_timestamp"] = possible_choice['timestamp']
    
    result_row["answer_timestamp"] = _hit["answer_timestamp"]
    result_row["id"] = update["id"]
    result_row["user_id"] = update["from"]["id"]
    result_row["first_name"] = update["from"]["first_name"]
    result_row["last_name"] = update["from"]["last_name"]
    result_row["username"] = update["from"]["username"]
    result_row["language_code"] = update["from"]["language_code"]
    result_row["chat_type"] = update["chat_type"]
    result_row["query"] = update["query"]
    result_row["candidates"] = _hit["template_options"]
    
    rows.append(result_row)

In [33]:
logs_df = pd.DataFrame(rows)

In [34]:
logs_df.head()

,answer_timestamp,id,user_id,first_name,last_name,username,language_code,chat_type,query,candidates,template_choice,choice_timestamp
0,2022-02-08T18:36:13+00:00,1187924786621411208,276585292,Daniel,Kovalenko,daniel_kovalenko,en,private,hellow,[xlq3a8d03gf61.jpg__775d3d68-8f5d-4825-b176-4a...,NaN,NaN
1,2022-02-08T18:36:13+00:00,1187924786233751295,276585292,Daniel,Kovalenko,daniel_kovalenko,en,private,hellow wo,[zyrdwm2tizl61.jpg__90d5563c-b8ad-4493-92fb-24...,NaN,NaN
2,2022-02-08T18:36:08+00:00,1187924787750078202,276585292,Daniel,Kovalenko,daniel_kovalenko,en,private,qweqweqweqwe,[eo2kkfhhl4l71.jpg__aa200045-f3a0-490c-95cc-02...,NaN,NaN
3,2022-02-08T18:36:14+00:00,1187924787098359127,276585292,Daniel,Kovalenko,daniel_kovalenko,en,private,hellow world,[51wqon12m9771.png__ccbbe43e-00ce-4a57-9c52-9e...,51wqon12m9771.png__ccbbe43e-00ce-4a57-9c52-9ed...,2022-02-08T18:36:16+00:00
4,2022-02-08T18:55:48+00:00,1187924784902022383,276585292,Daniel,Kovalenko,daniel_kovalenko,en,private,qw,[qbo5srpf8h371.png__3955c1ea-969e-494f-b188-81...,qbo5srpf8h371.png__3955c1ea-969e-494f-b188-816...,2022-02-08T18:55:58+00:00
